In [1]:
# import packages
import os, json
import glob
import pandas as pd
import numpy as np

from tqdm.notebook import trange, tqdm

In [2]:
paths = [['arantina', 'Arantina'], ['coqueiral', 'Coqueiral'], ['cristais', 'Cristais'], ['ijaci', 'Ijaci'], ['olaria', 'Olaria'], ['passa-vinte', 'Passa Vinte'], ['pirapetinga', 'Pirapetinga'], ['sao-bento-abade', 'São Bento Abade']]

In [4]:
folder = '../mp-ufmg-ner/M01-master/data/'
for file_path, file_city in paths:
    path_to_json = folder+file_path
    json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

    jsons_data = pd.DataFrame(columns=['city', 'sentences', 'src_file', 'file', 'timestamp'])
    for index, js in enumerate(json_files):
        with open(os.path.join(path_to_json, js), encoding='utf-8') as json_file:
            json_text = json.load(json_file)

            src_file = json_text['src_file']
            file = json_text['file']
            timestamp = json_text['timestamp']
            city = file_city
            
            try:
                sentences = json_text['sentences']
            except Exception as e:
                continue
            
            jsons_data.loc[index] = [city, sentences, src_file, file, timestamp]

    entities_dict = {}
    for idx, entry in tqdm(jsons_data.iterrows(), total=jsons_data.shape[0], desc = f'sentences in {file_city}'):
        d = json.dumps(entry['sentences'])
        sentences = json.loads(d)

        file_dict = {}
        file_dict['city'] = entry['city']
        file_dict['file'] = entry['file'].replace(f"/dados01/workspace/ufmg.f01dcc/m03/business_understanding/mp-ufmg-ner/M01-master/./data/{file_path}/", "").replace(".json", "")

        l_e = []
        entities_list = []
        for s in sentences:
            ds = json.dumps(s['entities'])
            entities = json.loads(ds)      
            for e in entities:
                start = e['start']
                end = e['end']
                label = e['label']
                entity = e['entity']
                l_e.append([label, entity])
                entities_list.append(label)

        file_dict['named_entities'] = l_e
        file_dict['entities'] = entities_list
        entities_dict[idx] = file_dict

    df = pd.DataFrame(entities_dict)
    df = df.T
    df.head()
    df.to_csv(f'./NER_results/{file_city}.csv', sep='\t', encoding='utf-8', index=False)

sentences in Arantina: 0it [00:00, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: './NER_results/Arantina.csv'